## 装饰器进阶-类装饰器-多重装饰器-惰性函数装饰器

在之前写过的文章中，我们有介绍过普通的装饰器知识，当然这种内容在网上一搜一大片，我敢说网上 90% 关于装饰器的知识都是比较入门的，今天，我们介绍一些较进阶的装饰器知识。

主要有以下三部分构成，难度也是依次递增:
- 类装饰器
- 多重装饰器
- 惰性函数装饰器

### 面对对象，我无所不能 —— **类装饰器**

In [7]:
import time

class ClassDecorator(object):
    
    
    def __init__(self, func):
        
        self._func = func
        
    def __call__(self, *args, **kw):
        start = time.time()
        res = self._func(*args, **kw)
        print('the func\'s excecute time is', time.time() - start)
        

In [8]:
@ClassDecorator
def func(obj):
    
    time.sleep(1)
    print(obj)

In [9]:
func("i am a func")

i am a func
the func's excecute time is 1.002049207687378


理解类装饰器，首先理解 “一个中心， 两个基本点”，中心就是你得知道装饰器的原理就是将一个对象A(类或者函数)传递给另一个对象B(类或者函数)，然后在 Ｂ 中对　Ａ　进行调用。两个基本点分别是，类由　\_\_init\_\_ 初始化的; 双下划线　\_\_call\_\_ 　函数的意义以及执行顺序。本文主要讲装饰器相关的内容，所以对这些细节就不再详细解释了。有兴趣的读者可以参考以下文章：

- 装饰器的基本使用方法
- [init 和 call 的介绍](https://stackoverflow.com/questions/9663562/what-is-the-difference-between-init-and-call-in-python)



再说类装饰器的优点，那就是 面对对象的时候，存在无限可能。在以上例子中，稍加修改：

In [10]:
class ClassDecorator2(object):
    
    
    def __init__(self, func):
        
        self._func = func
        self.start_call()
        
    def __call__(self, *args, **kw):
        start = time.time()
        res = self._func(*args, **kw)
        print('the func\'s excecute time is', time.time() - start)
        
    def start_call(self):
        
        print("the class decorator has been initiated")
        

In [12]:
@ClassDecorator2
def func2(obj):
    
    time.sleep(1)
    print("func2's obj is", obj)

the class decorator has been initiated


In [13]:
func2(123)

func2's obj is 123
the func's excecute time is 1.0017757415771484


看到了么，我们可以在函数还没调用的时候就可以执行一些操作。比如说当你执行某个函数之前，需要预先加载一些环境，使用类装饰器就可以非常优雅的做到这一点。
当然，这是只是抛砖引玉，更多更有趣的玩法还需要大家一起挖掘。

### 一切皆可被装饰 —— **多重装饰器**。

In [28]:
# 首先我们构造两个普通的装饰器

def decorator1(func):
    
    def inner(*args, **kw):
        
        r = func(*args, **kw)
        print("decorator1's result is", r)
        return "<i>" + r + "<i>"
    return inner


def decorator2(func):
    
    def inner(*args, **kw):
        
        r = func(*args, **kw)
        print("decorator2's result is", r)
        return "<dev>" + r + "<dev>"
    return inner

In [29]:

@decorator2
@decorator1
def func3(obj):
    return str(obj)
    

In [30]:
func3(345)

decorator1's result is 345
decorator2's result is <i>345<i>


'<dev><i>345<i><dev>'

从打印看，我们可以清楚的看到函数的执行顺序，首先装饰器 decorator1 作用于 func3 函数，然后装饰器 decorator2 作用于前两者的结果。这里需要明白，装饰器说白了，就是一个函数或者实例，而前者都可以被装饰。理解到这一点，就是一百重装饰器你也不用发虚了。

### 函数，你慢一点 —— **惰性函数装饰器**

以上两部分，相信对于已经会基本装饰器的玩家来说，理解起来不是难事，接下来要讲的是 惰性函数装饰器(我自己命名的，实在不知道怎么官方的描述)。
这类装饰器返回被装饰函数的函数地址，这样让开发者随心所欲的控制传参。

In [46]:
def decorator3(func):
    
    def outer(**outer_kw):
        
        def inner(**inner_kw):
            
            outer_kw.update(inner_kw)
            r = func(**outer_kw)
            return r
        
        return inner
    
    return outer

In [47]:
@decorator3
def func4(k=1, v=2):
    
    print("k is {0}, v is {1}".format(k, v))

In [48]:
func4()

<function __main__.decorator3.<locals>.outer.<locals>.inner>

In [49]:
func4()(k=100, v=200)

k is 100, v is 200


看到这里，我感觉大部分同学应该是懵逼的，不瞒大家说，我第一次见到这个代码的时候，先是懵逼，然后是惊叹，最后给作者献上了膝盖。原来装饰器还可以这么用，真是太奇妙了。

我们来解析一下这个装饰器的构造，为了方便起见，我们添加一些打印来让装饰器的执行变得清晰起来。

In [52]:
def decorator3(func):
    
    def outer(**outer_kw):
        
        def inner(**inner_kw):
            
            outer_kw.update(inner_kw)
            r = func(**outer_kw)
            print("decorator return finnal result")
            return r
        
        print("decorator return inner")
        return inner
    
    print("decorator return outer")
    return outer

In [61]:
@decorator3
def func4(k=1, v=2):
    
    print("k is {0}, v is {1}".format(k, v))

decorator return outer


In [62]:
new_f = func4()
print(new_f)

decorator return inner
<function decorator3.<locals>.outer.<locals>.inner at 0x7fae40515400>


In [64]:
r = new_f(k=100, v=200)

k is 100, v is 200
decorator return finnal result


可以清楚的看到， func4() 的调用结果是装饰器内部的 inner 函数地址，然后 new_f(k=100, v=200) 先是更新了 k,v 的值然后再进行最终的调用，即打印出已经更新的值。

这种装饰器的优点在于，你可以对函数进行两次赋值，然后再进行调用，在实际的工程项目中可能会用到这种设计。

最后，祝大家看完之后会有收获。